In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle
import re

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from lxml import etree
from collections import defaultdict
from pprint import pprint
from config.assistant import *
from config.site_config import *
from tqdm import tqdm

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

request_headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'),
}

In [2]:
with open(filter_list_company_name_path) as f:
    filter_list_company_name = f.read().splitlines()
with open(filter_list_company_name_part_path) as f:
    filter_list_company_name_part = f.read().splitlines()

with open(filter_list_posting_region_path) as f:
    filter_list_posting_region = f.read().splitlines()
with open(filter_list_posting_name_part_path) as f:
    filter_list_posting_name_part = f.read().splitlines()

In [3]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

5935


### 1. 필터링 - 회사명에 부분적으로 포함되는 단어

In [4]:
for filtering_word in filter_list_company_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['company']:
                if job_list['status'] == 'wait':
                    job_list['status'] = 'close'
                    nCnt += 1

    print(filtering_word, nCnt)

학원 0
스터디 0
교육원 0
교육센터 0
평생교육원 0
필라테스 0
아카데미 0
병원 0
부동산 0
중개 0
건축사 0
의원 0
결혼 0
출판 0
손해사정 0
축구 0
스쿨 0
학교 0
산부인과 0
주얼리 0
부티크 0
복지관 0
스투디오 0
에스씨에이게임 0
렌트카 0
스타트업 0
의료센터 0
웨딩 0
에이치알 0
테라로사 0
성형외과 0
변호사 0


### 2. 필터링 - 공고명에 부분적으로 포함되는 단어

In [5]:
for filtering_word in filter_list_posting_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['title']:
                if job_list['status'] == 'wait':
                    job_list['status'] = 'close'
                    nCnt += 1
                    
    print(filtering_word, nCnt)

영업관리 0
매장관리 0
고객 0
사무보조 0
마케팅 0
마케터 0
경영지원 0
NodeJs 0
Nestjs 0
부동산 0
영업사원 0
자재관리 0
회계 0
재무 0
총무 0
영상의학실 0
유지보수 0
고객응대 0
텔레마케터 0
아웃바운드 0
국비지원 0
화장품 0
디자이너 0
인플루언서 0
MD 0
AMD 0
온라인 0
네일리스트 0
웹디자인 0
기술영업 0
프론트엔드 0
회로설계 0
백엔드 0
강사 0
경리 0
사무행정 0
보험설계사 0
유튜브 0
유투브 0
국비무료 0
국비지원 0
수행기사 0
행정보조 0
구축사업 0
골프장 0
영업지원 0
Node.js 0
node.js 0
방송장비 0
물류팀 0
물류관리 0
창고관리 0
정보보안 0
품질경영 0
코스메틱 0
자동차튜닝 0
시각디자인 0
악세사리 0
업무제휴 0
무료 0
영상제작 0
생산관리 0
인사팀장 0
자동차영업 0
생산팀 0
창업센터 0
가맹점 0


In [6]:
# JSON 파일 쓰기
tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

### 3. 전체현황 - 회사명에 부분적으로 단어 포함

In [4]:
for filtering_word in filter_list_company_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['company']:
                    nCnt += 1

    print(filtering_word, nCnt)

학원 177
스터디 10
교육원 18
교육센터 7
평생교육원 3
필라테스 5
아카데미 194
병원 38
부동산 36
중개 45
건축사 7
의원 42
결혼 12
출판 8
손해사정 5
축구 2
스쿨 16
학교 64
산부인과 2
주얼리 5
부티크 1
복지관 2
스투디오 1
에스씨에이게임 34
렌트카 1
스타트업 4
의료센터 3
웨딩 6
에이치알 65
테라로사 7
성형외과 18
변호사 2


### 4. 하나씩 확인 - 회사명에 부분적으로 단어 포함

In [48]:
filtering_word = "변호사"
ans_list = []
for company_idx, company_title in enumerate(job_group):
    for job_list in job_group[company_title]:
        if filtering_word in job_list['company']:
            ans_list.append(company_title)
print(filtering_word)
print("검출개수: " + str(len(ans_list)))
ans_list = list(set(ans_list))
print("중복제거: " + str(len(ans_list)))
print("----------------------")
for ans in ans_list:
    print(ans)

변호사
검출개수: 2
중복제거: 1
----------------------
변호사이원후법률사무소


### 5. 전체현황 - 공고명에 부분적으로 단어 포함

In [5]:
for filtering_word in filter_list_posting_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['title']:
                    nCnt += 1
                    
    print(filtering_word, nCnt)

영업관리 49
매장관리 13
고객 171
사무보조 100
마케팅 1035
마케터 399
경영지원 277
NodeJs 1
Nestjs 1
부동산 75
영업사원 18
자재관리 10
회계 196
재무 82
총무 121
영상의학실 1
유지보수 125
고객응대 4
텔레마케터 1
아웃바운드 11
국비지원 7
화장품 130
디자이너 601
인플루언서 13
MD 413
AMD 91
온라인 510
네일리스트 1
웹디자인 69
기술영업 98
프론트엔드 163
회로설계 11
백엔드 271
강사 199
경리 54
사무행정 10
보험설계사 25
유튜브 47
유투브 1
국비무료 9
국비지원 7
수행기사 2
행정보조 2
구축사업 14
골프장 1
영업지원 77
Node.js 12
node.js 3
방송장비 3
물류팀 14
물류관리 17
창고관리 4
정보보안 32
품질경영 7
코스메틱 31
자동차튜닝 3
시각디자인 10
악세사리 2
업무제휴 3
무료 28
영상제작 13
생산관리 31
인사팀장 2
자동차영업 2
생산팀 11
창업센터 1
가맹점 5
